In [14]:
from transformers import BertModel, BertTokenizerFast

alephbert_tokenizer = BertTokenizerFast.from_pretrained('.')
alephbert = BertModel.from_pretrained('.')

# if not finetuning - disable dropout
alephbert.eval()



Some weights of the model checkpoint at . were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at . and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(52000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [5]:
import torch
import numpy as np

encoded_review = alephbert_tokenizer.encode_plus(
"[MASK] הלכתי אתמול",
return_tensors='pt',
)

In [13]:
from transformers import pipeline
alephbert.train
# Using custom model/tokenizer as str
# pipeline('ner', model='alephbert', tokenizer='alephbert_tokenizer')

<bound method Module.train of BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(52000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=

In [236]:
output[0].shape

torch.Size([1, 3, 768])

In [4]:
encoded_review['input_ids']

tensor([[   4, 7349, 3374]])

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

input_ids = encoded_review['input_ids'].to(device)
output = alephbert(input_ids)
features = output[0][:,0,:]
alephbert.pooler


BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)

In [214]:
print(features)

tensor([[ 1.6944e+00,  6.7569e-01, -8.8485e-01,  5.0348e-01, -2.4454e-01,
          9.0961e-01, -5.4618e-02, -6.5201e-01, -8.8401e-01,  6.6211e-01,
         -1.1388e+00, -1.1096e-01,  4.3389e-01, -1.0060e+00, -2.5943e-01,
          3.5049e-01,  1.5409e-01, -5.2340e-01,  4.1203e-01,  1.0360e+00,
          3.2617e-01, -3.2374e-01,  2.2006e-01, -7.2341e-01, -3.6956e-01,
         -1.4279e+00,  7.0975e-01, -3.8945e-01,  3.8829e-02,  2.6607e-01,
          7.5181e-01,  6.2708e-01,  6.1021e-01,  2.6319e-01, -3.2765e-01,
          2.9902e-01,  3.0578e-01,  3.7556e-02,  4.8910e-01,  1.0862e+00,
          3.8748e-01,  5.9753e-01,  8.7701e-01, -1.5870e-01, -5.4889e-02,
          2.4085e-01, -8.0972e-01,  6.3523e-02,  6.1940e-01,  4.1628e-01,
         -7.1843e-01, -7.6617e-01,  3.7924e-01, -3.9165e-01,  1.8714e-01,
         -8.3567e-01,  5.7863e-01,  1.1041e-01, -1.7567e+00,  1.4040e+00,
         -9.3055e-01, -2.9361e-01, -1.6803e+00, -4.1506e-01,  3.1522e-01,
         -7.5162e-01,  6.2033e-01,  2.

          1.0918e+00,  8.3793e-01,  3.5238e-01]], grad_fn=<SliceBackward>)


In [7]:
_, prediction = torch.max(output[1], dim=1)
print(prediction)
print(f'Review text: {"שלום חבר"}')
print(f'Something  : {class_names[prediction]}')


tensor([424])
Review text: שלום חבר


NameError: name 'class_names' is not defined

In [16]:
alephbert_tokenizer._convert_id_to_token(742)

'🎁'

In [103]:
chr(555)

'ȫ'

In [165]:
torch.max(output[1])


tensor(0.9413, grad_fn=<MaxBackward1>)

In [176]:
torch.max(output[1],dim=1)

torch.return_types.max(
values=tensor([0.9413], grad_fn=<MaxBackward0>),
indices=tensor([555]))

In [18]:
output[1][0][0].shape

torch.Size([])

In [27]:
print(version("transformers"))

NameError: name 'version' is not defined

In [23]:
import transformers